In [1]:
import pandas as pd
import os
from functools import reduce

In [2]:
caracteristiques = pd.read_csv("data/caracteristiques.csv")
vehicules = pd.read_csv("data/vehicules.csv")
usagers = pd.read_csv("data/usagers.csv")
lieux = pd.read_csv("data/lieux.csv")

df_agg = reduce(lambda df1,df2: pd.merge(df1,df2,on='Num_Acc'), [caracteristiques, vehicules, usagers, lieux])

/var/folders/kf/rsvfsx611_3f7p2ctq98yhqm0000gn/T/ipykernel_7236/3375855187.py:1: DtypeWarning: Columns (6,7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  caracteristiques = pd.read_csv("data/caracteristiques.csv")
/var/folders/kf/rsvfsx611_3f7p2ctq98yhqm0000gn/T/ipykernel_7236/3375855187.py:4: DtypeWarning: Columns (2,7,8,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv("data/lieux.csv")


In [3]:
features = ['Num_Acc', 'an', "catv", 'agg', 'int', 'adr', 'code_insee',
            'catr','circ', 'nbv', 'vosp', 'prof', 'plan','infra']

In [4]:
df_agg_clean = df_agg[features]

In [5]:
df_agg_clean = df_agg_clean[df_agg_clean["an"]==2021]
df_agg_clean = df_agg_clean[df_agg_clean["catv"].isin([1, 50, 60])]

In [6]:
df_agg_clean.loc[df_agg_clean["adr"].isna(), "adr"] = df_agg_clean.loc[df_agg_clean["adr"].isna(), "Num_Acc"].astype(str)

In [7]:
df_agg_clean = df_agg_clean[['Num_Acc', 'agg',
                             'int', 'adr', 'code_insee','catr','circ', 
                             'nbv', 'vosp', 'prof', 'plan','infra']].drop_duplicates()

In [8]:
df_agg_clean["route"] = df_agg_clean["adr"] + df_agg_clean["code_insee"]

In [9]:
def delete_wong_values(df, column):
    df_wrong_column = df.groupby(['route'], as_index=False)[column].nunique()
    wrong_column = set(df_wrong_column[df_wrong_column[column]>=2]['route'])
    print("There is ", len(wrong_column), " wrong value for ", column)
    return df[~df["route"].isin(wrong_column)]

def select_wong_values(df, column):
    df_wrong_column = df.groupby(['route'], as_index=False)[column].nunique()
    wrong_column = set(df_wrong_column[df_wrong_column[column]>=3]['route'])
    print("There is ", len(wrong_column), " wrong value for ", column)
    return df[df["route"].isin(wrong_column)]

In [32]:
#select_wong_values(df_agg_clean, 'vosp').sort_values(by="route")

In [33]:
#list_col = ["agg", "catr", 'circ', 'nbv', 'vosp', 'infra', 'vosp']
#for col in list_col:
#    df_agg_clean = delete_wong_values(df_agg_clean, col)

In [34]:
#df_agg_clean[df_agg_clean["route"].isin(wrong_catr)].sort_values(by="route")

In [35]:
#df_agg_clean.loc[df_agg_clean["route"].isin(wrong_catr), "catr"].replace({3:4})

In [10]:
df_agg_clean.groupby(['route'], as_index=False)['Num_Acc'].nunique().describe()

,Num_Acc
count,5729.000000
mean,1.223250
std,0.999953
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,27.000000


In [19]:
df_agg_clean.groupby(['route','agg','int', 'adr', 'code_insee','catr','circ', 
                      'nbv', 'vosp', 'prof', 'plan','infra'], as_index=False)['Num_Acc'].nunique().describe()

,agg,int,catr,circ,nbv,vosp,prof,plan,infra,Num_Acc
count,6912.000000,6912.000000,6912.000000,6912.000000,6912.000000,6912.000000,6912.000000,6912.000000,6912.000000,6912.000000
mean,1.860388,2.579572,3.844184,1.672598,2.143808,0.494358,1.220775,1.230035,0.813802,1.013889
std,0.346609,2.169901,0.972282,0.872255,1.180869,0.851410,0.526411,0.597134,2.240669,0.127681
min,1.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
25%,2.000000,1.000000,3.000000,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,1.000000
50%,2.000000,2.000000,4.000000,2.000000,2.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,2.000000,3.000000,4.000000,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,2.000000,9.000000,9.000000,4.000000,12.000000,3.000000,4.000000,4.000000,9.000000,4.000000


In [13]:
def one_hot_encoder(dataframe:pd.DataFrame, column:list):
    """ 
    One Hot Encoding of the list of Column column in the dataframe dataframe filling Nan value with the most frequent value
    """
    dummies = pd.get_dummies(dataframe[column])
    return pd.concat([dataframe.drop(column, axis=1), dummies], axis=1)

In [16]:
dict_value_by_column = {"agg":{1:"hors_agglo" , 2:"en_agglo"},
                        
                        "int":{1:"hors_intersection", 2:"intersection", 3:"intersection", 4:"intersection",
                               
                              5:"intersection", 6:"giratoire", 7:"place", 8:"passage_a_niveau", 9:"intersection"},
                        "catr":{1:"autoroute", 2:"route_nationale", 3:"route_departementale", 4:"voie_communales",
                                5:"hors_reseau_public", 6:"stationnement_ouvert_a_circulation",
                                7:"route_metropole_urbaine", 9:"autre"},
                        
                        "circ":{-1:"non_renseigne", 1:"sens_unique", 2:"bidirectionnelle",
                                3:"chaussees_separees", 4:"voies_d_affectation_variable"},
                        
                        "vosp":{-1:"non_renseigne", 0:"sans_objet", 1:"piste_cyclable", 
                                2:"bande_cyclable", 3:"voie_reservee"},
                        
                        "prof":{-1:"non_renseigne", 1:"plat",2:"pente", 3:"sommet_de_cote", 4:"bas_de_cote"},
                        
                        "plan":{-1:"non_renseigne", 1:"partie_rectiligne", 2:"en_courbe_a_gauche",
                                3:"en_courbe_a_droite", 4:"en_s"}
                        
                        
                       }

In [18]:
dict_value_by_column["catr"][1]

'autoroute'